# 3 – Evaluation (Final)

Dieses Notebook ist für die **Auswertung** eines bereits trainierten
Zwei-Stufen-Experiments gedacht. Es lädt die gespeicherten JSON-
Ergebnisse (`two_stage__<EXP_ID>.json`) und erstellt tabellarische
Übersichten und Plots (z.B. Precision/Recall/F1, Confusion-Matrizen).


In [7]:
import sys
from pathlib import Path
import os

cwd = Path.cwd()
project_root = cwd
while not (project_root / 'src').is_dir():
    if project_root.parent == project_root:
        raise RuntimeError("Projektwurzel mit 'src' nicht gefunden.")
    project_root = project_root.parent

print('Erkannte Projektwurzel:', project_root)
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

os.chdir(project_root)
print('Arbeitsverzeichnis gesetzt auf:', Path.cwd())


Erkannte Projektwurzel: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project
Arbeitsverzeichnis gesetzt auf: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project


In [8]:
# Bitte EXP_ID auf ein bereits trainiertes Experiment setzen.
EXP_ID = 'hp_long_eod_flex_0_7_result'
assert EXP_ID != 'CHANGE_ME', 'Bitte EXP_ID oben setzen.'


In [9]:
import pandas as pd

# NOTE: Nicht überschreiben. Diese Zelle nutzt die EXP_ID aus der Zelle oben.
p = f"notebooks/results/final_two_stage/two_stage_final__{EXP_ID}_predictions.csv"
pred = pd.read_csv(p, parse_dates=["date"])

n_test_tage = len(pred)
n_trade_tage = int(pred["combined_pred"].isin(["up","down"]).sum())
n_ohne_trades = int((pred["combined_pred"] == "neutral").sum())

n_test_tage, n_trade_tage, n_ohne_trades


(223, 94, 129)

In [10]:
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')

results_dir = Path('notebooks') / 'results' / 'final_two_stage'
json_path = results_dir / f'two_stage_final__{EXP_ID}.json'
print('Lese Ergebnisse aus:', json_path)
with json_path.open('r', encoding='utf-8') as f:
    results = json.load(f)

config = results.get('config', {})
print('Config:', config)

metrics_path = results_dir / f'two_stage_final__{EXP_ID}_metrics.csv'
if metrics_path.is_file():
    metrics_df = pd.read_csv(metrics_path)
    display(metrics_df)
else:
    print('Keine Metrics-CSV gefunden unter:', metrics_path)


Lese Ergebnisse aus: notebooks/results/final_two_stage/two_stage_final__hp_long_eod_flex_0_7_result.json
Config: {'exp_id': 'hp_long_eod_flex_0_7_result', 'horizon_days': 15, 'up_threshold': 0.02, 'down_threshold': -0.02, 'strict_monotonic': False, 'max_adverse_move_pct': 0.004, 'hit_within_horizon': True, 'first_hit_wins': True, 'price_source': 'eodhd', 'dataset_path': 'data/processed/datasets/eurusd_news_training__hp_long_eod_flex_0_7_result.csv', 'feature_cols': ['intraday_range_pct', 'upper_shadow', 'lower_shadow', 'price_close_ret_1d', 'price_close_ret_5d', 'price_range_pct_5d_std', 'price_body_pct_5d_mean', 'price_close_ret_30d', 'price_range_pct_30d_std', 'price_body_pct_30d_mean', 'month', 'quarter', 'cal_dow', 'cal_day_of_month', 'cal_is_monday', 'cal_is_friday', 'cal_is_month_start', 'cal_is_month_end', 'hol_is_us_federal_holiday', 'hol_is_day_before_us_federal_holiday', 'hol_is_day_after_us_federal_holiday'], 'test_start': '2025-01-01', 'train_frac_within_pretest': 0.7, 'sig

,model,split,precision_pos,recall_pos,f1_pos
0,signal,train,0.746528,0.888430,0.811321
1,signal,val,0.388462,0.495098,0.435345
2,signal,test,0.267442,0.425926,0.328571
3,direction,train,0.709581,0.959514,0.815835
4,direction,val,0.505882,0.905263,0.649057
5,direction,test,0.744681,0.833333,0.786517


In [11]:
# PDF-Report mit dem bestehenden Report-Skript erzeugen
from pathlib import Path
import os
import sys
from scripts import generate_two_stage_report

# Matplotlib Cache in ein beschreibbares Verzeichnis legen (verhindert Warnungen)
os.environ.setdefault('MPLCONFIGDIR', '/tmp/mpl')

base_results_dir = Path('notebooks') / 'results'
final_results_dir = base_results_dir / 'final_two_stage'
final_results_dir.mkdir(parents=True, exist_ok=True)

pdf_path = final_results_dir / f'two_stage_final__{EXP_ID}_report.pdf'

sys.argv = ['', '--exp-id', EXP_ID, '--output', str(pdf_path)]
generate_two_stage_report.main()

print('[ok] Final-PDF gespeichert unter:', pdf_path)


[ok] Report gespeichert unter: notebooks/results/final_two_stage/two_stage_final__hp_long_eod_flex_0_7_result_report.pdf
[ok] Final-PDF gespeichert unter: notebooks/results/final_two_stage/two_stage_final__hp_long_eod_flex_0_7_result_report.pdf


In [12]:
# Optional: Vergleich EODHD vs Yahoo (Strategie B, Hebel 20)
#
# Voraussetzungen:
# - Beide Experimente wurden trainiert + evaluiert (Predictions-CSV existiert)
# - EXP_ID (dieses Notebook) ist dein EODHD-Experiment
#
# Hinweis: Das Vergleichsskript liegt im Archiv: archive/scripts/compare_experiments_pnl.py

import os
import subprocess
import sys
from pathlib import Path

os.environ.setdefault('MPLCONFIGDIR', '/tmp/mpl')
Path(os.environ['MPLCONFIGDIR']).mkdir(parents=True, exist_ok=True)

EXP_YAHOO = 'hp_long_final_yahoo'  # <- anpassen (Yahoo-Experiment)
EXP_EODHD = EXP_ID                 # <- dieses EODHD-Experiment

out_dir = Path('notebooks') / 'results' / 'final_two_stage' / 'pdf'
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / f'compare_{EXP_YAHOO}_vs_{EXP_EODHD}__B_pnl_lev20.pdf'

script_path = Path('archive') / 'scripts' / 'compare_experiments_pnl.py'
cmd = [
    sys.executable,
    str(script_path),
    '--exp-a', EXP_YAHOO,
    '--exp-b', EXP_EODHD,
    '--label-a', 'Yahoo',
    '--label-b', 'EODHD',
    '--strategy', 'B',
    '--leverage', '20',
    '--output', str(out_path),
]
subprocess.check_call(cmd)
print('[ok] Vergleich gespeichert unter:', out_path)


[ok] Vergleichsgrafik gespeichert: notebooks/results/final_two_stage/pdf/compare_hp_long_final_yahoo_vs_hp_long_eod_flex_0_7_result__B_pnl_lev20.pdf
[ok] Vergleich gespeichert unter: notebooks/results/final_two_stage/pdf/compare_hp_long_final_yahoo_vs_hp_long_eod_flex_0_7_result__B_pnl_lev20.pdf
